In [ ]:
import os
import shutil
import random
from typing import Dict, List

# --- Configuration ---
source_folder = r'/home/arc/EggProject/seg_base_project/source_folder' 
target_base_folder = 'yolo_egg_dataset_4'

# 데이터 분할 비율 (70:20:10)
SPLIT_RATIOS = {
    'train': 0.9,
    'val': 0.1,
}
# 지원되는 이미지 확장자에 대문자 확장자를 모두 포함합니다.
IMAGE_EXTENSIONS = ('.jpg', '.jpeg', '.png', '.bmp', '.JPG', '.JPEG', '.PNG', '.BMP') 
# --- End Configuration ---

def create_yolo_dataset_structure(
    source_folder: str, 
    target_base_folder: str, 
    split_ratios: Dict[str, float], 
    seed: int = 42
) -> None:
    
    if abs(sum(split_ratios.values()) - 1.0) > 1e-6:
        raise ValueError(f"분할 비율의 합계가 1.0이 아닙니다. 현재 합계: {sum(split_ratios.values())}")
    
    random.seed(seed)
    
    print(f"--- YOLO 데이터셋 분할 및 구조 생성 시작 (70:20:10) ---")
    print(f"원본 폴더: {source_folder}")
    print(f"대상 폴더: {target_base_folder}")
    
    all_files = os.listdir(source_folder)
    
    # 1. 파일 목록 디버그 출력 (문제 파악용)
    print("\n[디버그] 현재 폴더에서 인식된 파일 목록 (상위 5개):")
    for i, f in enumerate(all_files):
        if i < 5:
            # 파일 이름 양쪽의 공백을 제거합니다. (숨겨진 공백 처리)
            cleaned_f = f.strip() 
            print(f"  - 원본: '{f}', 클린: '{cleaned_f}'")
    print("----------------------------------------------------\n")

    # 2. 이미지 파일 필터링 및 클린 처리 (가장 중요한 부분)
    image_files = []
    
    for f in all_files:
        # 파일 이름 양쪽의 공백을 제거합니다.
        cleaned_f = f.strip()
        
        # 파일의 확장자를 얻습니다.
        _, ext = os.path.splitext(cleaned_f)
        
        # 파일의 확장자가 IMAGE_EXTENSIONS에 있는지 확인합니다.
        if ext in IMAGE_EXTENSIONS:
            image_files.append(cleaned_f)

    if not image_files:
        print(f"경고: 원본 폴더에서 이미지 파일({', '.join(IMAGE_EXTENSIONS)})을 찾을 수 없습니다. 작업을 중단합니다.")
        return

    # 3. 유효한 이미지-라벨 쌍 목록 생성
    valid_base_names = []
    for img_file in image_files:
        base_name = os.path.splitext(img_file)[0]
        # 라벨 파일 이름도 공백을 제거한 기본 이름에 .txt를 붙여서 확인합니다.
        label_file = base_name + '.txt'
        
        # 이미지와 TXT 라벨 파일이 모두 있는지 확인합니다.
        if os.path.exists(os.path.join(source_folder, label_file)):
            valid_base_names.append(base_name)
        else:
            print(f"주의: {img_file}에 해당하는 라벨 파일({label_file})이 없어 제외됩니다.")

    print(f"총 유효 데이터 쌍 수: {len(valid_base_names)}")
    if not valid_base_names:
        print("오류: 유효한 이미지/라벨 쌍이 없습니다. 작업을 중단합니다.")
        return

    # 4. 학습, 검증, 테스트 데이터 분할
    random.shuffle(valid_base_names)
    
    sets: Dict[str, List[str]] = {}
    current_idx = 0
    total_count = len(valid_base_names)

    for set_name, ratio in split_ratios.items():
        count = round(ratio * total_count)
        if set_name == 'test':
             count = total_count - current_idx
        
        sets[set_name] = valid_base_names[current_idx : current_idx + count]
        current_idx += count
        
        print(f"  -> {set_name.upper()} ({int(ratio*100)}%): {len(sets[set_name])}개 파일.")
        
    if current_idx < total_count:
        sets['train'].extend(valid_base_names[current_idx:])
        print(f"  -> Rounding remainder ({total_count - current_idx}개)를 TRAIN에 추가했습니다.")

    # 5. YOLOv8 타겟 폴더 구조 생성 및 파일 복사
    for set_name in sets.keys():
        os.makedirs(os.path.join(target_base_folder, 'images', set_name), exist_ok=True)
        os.makedirs(os.path.join(target_base_folder, 'labels', set_name), exist_ok=True)

    print("\n6. 이미지 및 라벨 파일 복사 중...")
    
    # 원본 파일명과 클린 파일명 간의 매핑을 위해 딕셔너리를 다시 만듭니다.
    cleaned_to_original = {f.strip(): f for f in all_files}

    for set_name, name_list in sets.items():
        for base_name in name_list:
            
            # base_name (확장자 없음, 공백 없음)을 사용하여 원본 파일명을 찾습니다.
            found_img_name = None
            found_label_name = None

            # 이미지 원본 파일 찾기
            for ext in IMAGE_EXTENSIONS:
                possible_clean_img_name = base_name + ext
                if possible_clean_img_name in cleaned_to_original:
                    found_img_name = cleaned_to_original[possible_clean_img_name]
                    break
            
            # 라벨 원본 파일 찾기
            possible_clean_label_name = base_name + '.txt'
            if possible_clean_label_name in cleaned_to_original:
                found_label_name = cleaned_to_original[possible_clean_label_name]

            if not found_img_name or not found_label_name:
                print(f"오류: {base_name}에 해당하는 이미지/라벨 파일의 원본 이름을 찾을 수 없습니다. 건너뜁니다.")
                continue

            # Source paths (원본 파일명 사용)
            img_src_path = os.path.join(source_folder, found_img_name)
            label_src_path = os.path.join(source_folder, found_label_name)
            
            # Destination paths (원본 파일명 사용)
            img_dest_path = os.path.join(target_base_folder, 'images', set_name, found_img_name)
            label_dest_path = os.path.join(target_base_folder, 'labels', set_name, found_label_name)
            
            try:
                shutil.copy2(img_src_path, img_dest_path)
                shutil.copy2(label_src_path, label_dest_path)
            except Exception as e:
                print(f"오류: {base_name} 파일 복사 중 문제 발생: {e}")

    print("\n🎉 데이터셋 준비 완료! 'yolo_egg_dataset' 폴더를 확인하세요.")


# ----------------------------------------------
# --- 함수 실행 ---
# ----------------------------------------------

if __name__ == '__main__':
    try:
        create_yolo_dataset_structure(
            source_folder=source_folder,
            target_base_folder=target_base_folder,
            split_ratios=SPLIT_RATIOS,
            seed=42
        )
    except Exception as e:
        print(f"최종 오류입니다 : {e}")